In [13]:
pip install requests requests-oauthlib

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install sec-cik-mapper

In [1]:
pip install pyally

     |████████████████████████████████| 77 kB 2.5 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
#This is a working example using requests_oauthlib

import inspect

from requests_oauthlib import OAuth1Session
import requests
import pandas as pd
import ally as pyally
import plotly.express as px
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing
import plotly.graph_objects as go
import config as cfg
from sec_cik_mapper import StockMapper
from pathlib import Path

pd.options.display.max_rows = None
pd.options.display.max_columns = None
pd.options.display.max_colwidth = None


/Users/derek/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.0)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:

params = {
    'ALLY_CONSUMER_SECRET':cfg.ally_config['consumer_secret'],
    'ALLY_CONSUMER_KEY':cfg.ally_config['consumer_key'],
    'ALLY_OAUTH_SECRET':cfg.ally_config['oauth_token_secret'],
    'ALLY_OAUTH_TOKEN':cfg.ally_config['oauth_token'],
    'ALLY_ACCOUNT_NBR':cfg.ally_config['account_nmbr'],
}
a = pyally.Ally(params)

In [3]:
o = pyally.Order.Order(
        buysell = 'buy',
        symbol = 'NOW', # case insensitive
        price = pyally.Order.Market(),
        time = 'day', #other options
        qty = 1
)

In [4]:
#Get a summary of this order in human-readable terms
print(str(o))
#'Side.Buy 8 units of "AAPL" TimeInForce.Day, Market'

# See a hypothetical execution info for this order,
#  without actually submitting the order
a.submit(o)

# Actually submit this order for execution
#  Returns the ID of this new order, but this parameter
#  also becomes a parameter of the 'o' object created earlier
o.orderid
#None

a.submit(o, preview=False)
#'SVI-12345678'

o.orderid
#'SVI-12345678'

(OType.Order) Side.Buy 1 units of "NOW" TimeInForce.Day, Market


ExecutionException: You have not funded your account yet. Please go to the Transfer Money tab to get started funding your account.

## Without pyally

In [44]:


def get_ally(opt, ckey, csec, otoken, otoksec):
    
    opts = ['toplosers','toppctlosers','topvolume','topactive','topgainers','toppctgainers']
    
    if opt == '':
        opt = 'topvolume'
       
    if opt not in opts:
        return 'error'
    
    consumer_key = ckey
    consumer_secret=csec
    oauth_token=otoken
    oauth_token_secret=otoksec

    test = OAuth1Session(consumer_key,
              client_secret=consumer_secret,
              resource_owner_key=oauth_token,
              resource_owner_secret=oauth_token_secret)
    url = 'https://devapi.invest.ally.com/v1/market/toplists/'+opt+'.json'
    r = test.get(url)
    data = r.json()
    
    return data

def get_ally_quote(opt, ckey, csec, otoken, otoksec):
     
    consumer_key = ckey
    consumer_secret=csec
    oauth_token=otoken
    oauth_token_secret=otoksec

    test = OAuth1Session(consumer_key,
              client_secret=consumer_secret,
              resource_owner_key=oauth_token,
              resource_owner_secret=oauth_token_secret)
    url = 'https://devapi.invest.ally.com/v1/market/quotes/'+opt+'.json'
    r = test.get(url)
    data = r.json()
    
    return data
            
def get_timeseries(symbol, fmp_key):
    #alphavantage.co API Key
    key = fmp_key
    url = 'https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol='+symbol+'&apikey='+key
    r = requests.get(url)
    data = r.json()
    return data
            
def format_timeseries(dat):
    dat = pd.DataFrame.from_dict(data['Weekly Time Series'], orient='index')
    dat.columns = ['open','high','low','close','vol']
    dat['DATE'] = pd.to_datetime(dat.index)

    fig = px.line(dat, x='DATE', y='close')
    
def buy():
    
  


In [47]:
ckey = 'consumer_key'
csec = 'consumer_secret'
otoken = 'oauth_token'
otoksec = 'oauth_token_secret'

tst = get_ally('toppctgainers', ckey, csec, otoken, otoksec)
tst = pd.DataFrame.from_dict(data['response']['quotes']['quote'])

In [48]:
tst.head(20)

,chg,chg_sign,last,name,pchg,pcls,rank,symbol,vl
0,4457.45,u,466237.45,BERKSHIRE HATHAWAY INC,0.97,461780.00,1,BRK'A,4376
1,155.390,u,5572.500,NVR INC.,2.87,5417.110,2,NVR,12090
2,52.80,u,2458.99,"AUTOZONE, INC.",2.19,2406.19,3,AZO,147013
3,35.84,u,1708.01,CHIPOTLE MEXICAN GRILL INC,2.14,1672.17,4,CMG,229592
4,30.3400,u,1530.0800,"METTLER-TOLEDO INTERNATIONAL, INC.",2.02,1499.7400,5,MTD,50941
5,29.690,u,702.370,CABLE ONE INC,4.41,672.680,6,CABO,60401
6,25.200,u,464.830,SERVICENOW INC,5.73,439.630,7,NOW,2096607
7,21.6863,u,1270.1600,DANAHER CORP,1.74,1248.4737,8,DHR.B,11029
8,19.730,u,689.000,GRAINGER (W.W.) INC.,2.95,669.270,9,GWW,206799
9,15.7000,u,1173.2000,WELLS FARGO & CO (NEW),1.36,1157.5000,10,WFC.L,15548
